In [22]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import xarray as xr
import pandas as pd
from scipy import stats as st

In [44]:
dataset_path = "data\\cdf\\tamsatDaily.v3.1-19930101-20231231-20241213_nir.nc"
data = xr.open_dataset(dataset_path)

In [45]:
data

<xarray.Dataset> Size: 7GB
Dimensions:     (lat: 256, lon: 320, time: 11322)
Coordinates:
  * lat         (lat) float32 1kB 13.88 13.84 13.8 13.76 ... 4.387 4.35 4.312
  * lon         (lon) float32 1kB 2.7 2.737 2.775 2.812 ... 14.59 14.62 14.66
  * time        (time) datetime64[ns] 91kB 1993-01-01 1993-01-02 ... 2023-12-31
Data variables:
    rfe         (time, lat, lon) float32 4GB ...
    rfe_filled  (time, lat, lon) float32 4GB ...
Attributes:
    Conventions:  CF-1.6
    CreatedBy:    EDAL Libraries
    MoreInfo:     https://github.com/Reading-eScience-Centre/edal-java

In [46]:
ds_pr = data['rfe']
ds_RR_WestAfrica = ds_pr.sel(lon=slice(-20,15), lat=slice(0,15), time=slice('1993', '2023'))

In [47]:
def spi(ds, thresh, dimension):
    #ds - data ; thresh - time interval / scale; dimension - dimension as a string
    
    #Rolling Mean / Moving Averages
    ds_ma = ds.rolling(time = thresh, center=False).mean(dim=dimension)
    
    #Natural log of moving averages
    ds_In = np.log(ds_ma)
    ds_In = ds_In.where(np.isinf(ds_In) == False) #= np.nan  #Change infinity to NaN
    
    ds_mu = ds_ma.mean(dimension)
    
    #Overall Mean of Moving Averages
    ds_mu = ds_ma.mean(dimension)
    
    #Summation of Natural log of moving averages
    ds_sum = ds_In.sum(dimension)
    
    #Computing essentials for gamma distribution
    n = ds_In[thresh-1:, :, :].count(dimension)                  #size of data
    A = np.log(ds_mu) - (ds_sum/n)             #Computing A
    alpha = (1/(4*A))*(1+(1+((4*A)/3))**0.5)   #Computing alpha  (a)
    beta = ds_mu/alpha                         #Computing beta (scale)
    
    #Gamma Distribution (CDF)
    gamma_func = lambda data, a, scale: st.gamma.cdf(data, a=a, scale=scale)
    gamma = xr.apply_ufunc(gamma_func, ds_ma, alpha, beta)
    
    #Standardized Precipitation Index   (Inverse of CDF)
    norminv = lambda data: st.norm.ppf(data, loc=0, scale=1)
    norm_spi = xr.apply_ufunc(norminv, gamma)  #loc is mean and scale is standard dev.
    
    return ds_ma, ds_In , ds_mu, ds_sum,n, A, alpha, beta, gamma, norm_spi

In [48]:
i=3
data['spi_3'] = spi(ds_RR_WestAfrica,i,'time')[9]

C:\Users\IITA-GISguest2\AppData\Local\Temp\ipykernel_13388\1143478538.py:5: DeprecationWarning: Reductions are applied along the rolling dimension(s) '['time']'. Passing the 'dim' kwarg to reduction operations has no effect.
  ds_ma = ds.rolling(time = thresh, center=False).mean(dim=dimension)


In [ ]:
data['spi_3'].sel(time='1993').plot(cmap='RdBu', col='time', col_wrap=4, vmin=-2.5, vmax=2.5)
plt.ylim(0,15)
plt.xlim(-20,15)